In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import math
from collections import Counter
import tkinter as tk
from tkinter import ttk

In [2]:
data = pd.read_csv("/Users/faressaad/Downloads/Assignment 3/diabetes.csv")
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
def custom_discretize_numerical_column_uniform(column, num_bins=5):
    min_val = column.min()
    max_val = column.max()
    bin_width = (max_val - min_val) / num_bins

    # Assign bin labels based on uniform intervals
    discretized_column = ((column - min_val) / bin_width).astype(int)

    return discretized_column

def discretize_dataframe_uniform(df,threhold=10, num_bins=5):

    for col in df.columns:
        if df[col].dtype == 'object' or len(df[col].unique()) < threhold:
            # Column is categorical or has too many unique values, skip discretization
            continue
        else:
            # Column is numerical and suitable for discretization
            df[col + '_bins'] = custom_discretize_numerical_column_uniform(df[col], num_bins=num_bins)
            df.drop(columns=[col], inplace=True)

    return df

In [4]:
def entropy(labels):
    n = len(labels)
    if n <= 1:
        return 0
    
    label_counts = Counter(labels)
    entropy_value = 0
    for count in label_counts.values():
        p = count / n
        entropy_value -= p * math.log2(p)
    
    return entropy_value

In [5]:
def information_gain(data, split_attribute_name, target_name):
    """ Calculate the information gain given a dataset, splitting attribute, and target attribute. """
    # Calculate the total entropy before splitting
    total_entropy = entropy(data[target_name])

    # Calculate the weighted entropy after splitting by split_attribute_name
    # Collect unique values of the split attribute
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)

    # Calculate the sum of the entropies for each subset of data
    weighted_entropy = sum([(counts[i] / sum(counts)) *
                            entropy(data.where(data[split_attribute_name] == val).dropna()[target_name])
                            for i, val in enumerate(vals)])

    # Calculate the information gain
    information_gain = total_entropy - weighted_entropy
    return information_gain

In [6]:
def maxInfoGain(data,target):
    max=-1
    name="not yet"
    for col in data.columns.tolist():
        if col != target:
            temp=information_gain(data, col, target)
            if(max<temp):
                max=temp
                name=col
            
    return name

In [7]:
class Node():
    def __init__(self, data, name):
        self.name=name
        self.data=data
        self.cat =data[name].unique()
        self.children={}
        self.leaf=0
        self.label=None
    

In [8]:
class Decision_Tree():
    def __init__(self, data, target):
        self.root=None
        self.data=data
        self.col=data.columns.tolist()
        self.target=target
    def dt(self):
        self.root=self.build(self.data,self.target)
    
    def build(self,data,target):
        
        label_counts = Counter(data[target])
        if(len(label_counts)==1 or len(data.columns.tolist())<2 ):
            #print("fes"+str(label_counts))
            most_common_label = label_counts.most_common(1)[0][0]
            new_node = Node(data,"diabetes")#NODE(weather)
            new_node.leaf=1
            new_node.label=most_common_label
            return new_node

        else:
            #print("fesfes")
            selected=maxInfoGain(data,target)#weather
            new_node = Node(data,selected)#NODE(weather)
            for cat in new_node.cat:#cold hot warm
                data_col=data[data[selected]==cat].drop(columns=selected)
                new_node.children[str(cat)]=self.build(data_col,target)
        return new_node

    def print_tree(self):
        self.print_children(self.root.children,self.root.name)

    def print_children(self,children,name):
        print(name)#weather
        for key, value in children.items():
            if(value.leaf==0):
                print("cat --> "+str(key))#cold
                self.print_children(value.children,value.name)#heat_disease
            else:
                print("leaf  "+str(value.label))
                

    def test(self,row):
        node = self.root
        while(node.leaf==0):
            name=node.name#weather
            cat=row.loc[name]
            node= node.children[str(cat)]

        return node.label

    def accuracy(self,test):
        sum=0
        for index, row in test.iterrows():
            actual=row.loc[self.target]
            print(f"index {index} : label is {actual}")
            pred=self.test(row)
            print(f"index {index} : pred is {pred}")
            if(actual==pred):
                sum+=1
                
        return sum/len(test)

In [9]:
#dt=Decision_Tree(data[["gender","smoking_history","heart_disease","diabetes"]],"diabetes")

In [10]:
#print(data.columns.tolist())

In [11]:
#dt.dt()

In [12]:
#dt.root.name

In [13]:
#dt.root.children

In [14]:
#dt.root.children["never"].name

In [15]:
#dt.root.children["never"].children

In [16]:
#dt.root.children["never"].children["1"].name

In [17]:
#dt.root.children["never"].children["1"].children

In [18]:
#dt.root.children["never"].children["1"].children["Male"].label

In [19]:
#dt.print_tree()

In [20]:
#dt.accuracy(data[:100])

In [ ]:
def on_submit():
    data = pd.read_csv("/Users/faressaad/Downloads/Assignment 3/diabetes.csv")
    df=discretize_dataframe_uniform(data)
    training_size = int(training_size_entry.get())
    test_size_1 = int(test_size_entry.get())
    
    total_rows = len(df)
    train_size = int(0.75 * total_rows)  # 75% for training
    test_size = total_rows - train_size  # Remaining 25% for testing

    # Split the DataFrame into training and testing sets
    df_train = df.iloc[:train_size]   # First 75% for training
    df_test = df.iloc[train_size:]   
    
    dt=Decision_Tree(df,"diabetes")
    dt.dt()
    accuracy=20
    accuracy = dt.accuracy(df_test[:test_size_1])
    result_label.config(text=f"Accuracy: {accuracy:.2f}%")

# Create main application window
root = tk.Tk()
root.title("Accuracy Calculator")

# Create input widgets
training_label = ttk.Label(root, text="Training Set Size:")
training_label.grid(row=0, column=0, padx=10, pady=5, sticky="e")
training_size_entry = ttk.Entry(root, width=10)
training_size_entry.grid(row=0, column=1, padx=10, pady=5)

test_label = ttk.Label(root, text="Test Set Size:")
test_label.grid(row=1, column=0, padx=10, pady=5, sticky="e")
test_size_entry = ttk.Entry(root, width=10)
test_size_entry.grid(row=1, column=1, padx=10, pady=5)

submit_button = ttk.Button(root, text="Calculate Accuracy", command=on_submit)
submit_button.grid(row=2, column=0, columnspan=2, pady=10)

result_label = ttk.Label(root, text="", font=("Helvetica", 12, "bold"))
result_label.grid(row=3, column=0, columnspan=2, pady=10)

# Start the main event loop
root.mainloop()


index 75000 : label is 0
index 75000 : pred is 0
index 75001 : label is 0
index 75001 : pred is 0
index 75002 : label is 1
index 75002 : pred is 1
index 75003 : label is 0
index 75003 : pred is 0
index 75004 : label is 0
index 75004 : pred is 0
index 75005 : label is 0
index 75005 : pred is 0
index 75006 : label is 0
index 75006 : pred is 0
index 75007 : label is 0
index 75007 : pred is 0
index 75008 : label is 1
index 75008 : pred is 1
index 75009 : label is 0
index 75009 : pred is 0
index 75010 : label is 0
index 75010 : pred is 0
index 75011 : label is 0
index 75011 : pred is 0
index 75012 : label is 0
index 75012 : pred is 0
index 75013 : label is 0
index 75013 : pred is 0
index 75014 : label is 0
index 75014 : pred is 0
index 75015 : label is 0
index 75015 : pred is 0
index 75016 : label is 0
index 75016 : pred is 0
index 75017 : label is 1
index 75017 : pred is 0
index 75018 : label is 0
index 75018 : pred is 0
index 75019 : label is 0
index 75019 : pred is 0
index 75020 : label 